In [6]:
# MEDICIONES SOBRE TIERRA

In [5]:
import numpy as np
from scipy import signal
from scipy.fftpack import fft, fftfreq, fftshift
from scipy import stats

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

from varname import nameof
import os

In [29]:
%run check_import.ipynb
%run codigos/to_baseband.ipynb
%run codigos/FFT_ordenada.ipynb
%run codigos/to_dB.ipynb
%run codigos/mv_to_ep.ipynb
%run codigos/R_Fresnel.ipynb

In [30]:
#-----------------------------------
## Parámetros globales de la antena
#-----------------------------------

Fs = 23.328e9 #Frecuencia de muestreo [Hz] 
Ts = 1/Fs
BW = 1.5e9 
corte = 150
Nfft= 1024 # nro de frecs -> conviene que sea multiplo de 2 para calcular la FFT de manera eficiente. 
#Tiene que ser mayor a la longitud de la señal (285 para 2m, 439 para 3m)

#------------------------------
#Elegir frecuencia portadora
#------------------------------

#portadora = 'Fc7'
portadora = 'Fc8'

if portadora == 'Fc7':
    fc = 7.290e9 
    Fc = '7.29GHz'
elif portadora == 'Fc8':
    fc = 8.748e9
    Fc = '8.74GHz'
else:
    print('Error: portadora debe ser Fc7 o Fc8')


#------------------------------
# Elegir sistema operativo
#------------------------------
system = 'ubuntu'
#system = 'windows'


## Mediciones

In [31]:
date = ['mediciones_05_27', 'mediciones_06_09', 'mediciones_07_15', 'mediciones_08_22']
path = []

for i in range(len(date)):
    path.append(os.path.join(sys.path[0],date[i]))

print(path)



['c:\\Users\\User\\Documents\\Laboratorio\\Profesores\\Carolina Castedo\\facu\\codigos\\tesis\\mediciones_05_27', 'c:\\Users\\User\\Documents\\Laboratorio\\Profesores\\Carolina Castedo\\facu\\codigos\\tesis\\mediciones_06_09', 'c:\\Users\\User\\Documents\\Laboratorio\\Profesores\\Carolina Castedo\\facu\\codigos\\tesis\\mediciones_07_15', 'c:\\Users\\User\\Documents\\Laboratorio\\Profesores\\Carolina Castedo\\facu\\codigos\\tesis\\mediciones_08_22']


In [32]:
S1_may = check_import(path[0], 'S1', portadora)
S1_jun = check_import(path[1], 'S1', portadora)
S1_jul = check_import(path[2], 'S1', portadora)
S1A_ag = check_import(path[3], 'S1A', portadora)
S1B_ag = check_import(path[3], 'S1B', portadora)

print(np.shape(S1_may))
print(np.shape(S1_jun))
print(np.shape(S1_jul))
print(np.shape(S1A_ag))
print(np.shape(S1B_ag))

#https://researchdatapod.com/how-to-solve-python-attributeerror-str-object-has-no-attribute-read/#:~:text=DSA-,How%20to%20Solve%20Python%20AttributeError%3A%20'str',object%20has%20no%20attribute%20'read'&text=This%20error%20typically%20occurs%20when,or%20reading%20a%20file%20object.

AttributeError: 'str' object has no attribute 'readlines'

In [27]:
A = os.path.join(path[0],'S1')
print(A)
# open(os.path.join(path,'S1',portadora,'1','1'+'.txt').readlines())

c:\Users\User\Documents\Laboratorio\Profesores\Carolina Castedo\facu\codigos\tesis\mediciones_05_27\S1


In [65]:
def transformada(señal,fc,Nfft,Ts):
    avg  = np.mean(señal[:,:,:,1:],axis=3)

    indices = np.shape(avg)
    N_h = indices[0]
    N_med = indices[1]

    Scal_fft = np.zeros((N_h,N_med,Nfft),dtype=complex)
    frec_fft = np.zeros((N_h,Nfft),dtype=complex)


    ## Transformo Fourier
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    Deltaf = 1.75e9
    f_min = fc-Deltaf
    f_max = fc+Deltaf

    for i in range(N_h):

        ## Transformo Fourier S_cal y grafico
        for j in range(N_med):
            Scal_fft[i,j,:] = FFT_ordenada(avg[i,j,:],Nfft,Ts)[1]
        frec_fft[i,:] = FFT_ordenada(avg[i,0,:],Nfft,Ts)[0]

        # print(np.shape(frec_fft))
        # print(np.shape(Scal_fft))

        #promedio las 10 mediciones Scal
        Scal_fft_avg = np.zeros((N_h,Nfft), dtype=complex)
        for j in range(Nfft):
            Scal_fft_avg[i,j] = np.mean(Scal_fft[i,:,j])
        
        # print(np.shape(Scal_fft_avg))

        #me quedo con las frecuencias positivas
        f_pos = []

        f_pos = frec_fft[i,(frec_fft[1]>f_min) & (frec_fft[1]<f_max)]
        n_frecs = len(f_pos)

        print(n_frecs)
        start = int(np.min(np.where(frec_fft>f_min)))
        end = int(np.max(np.where(frec_fft<f_max))+1)
        f_norm = f_pos/1e9 # frec >0 normalizada en GHz

        Scal_fpos = Scal_fft_avg[i,start:end]

        print(np.shape(Scal_fpos))

        ## Grafico
        fig , ax = plt.subplots(nrows=1, ncols=2, figsize=(16,12))

        ax[0].plot(f_norm,np.real(Scal_fpos[i,:]))
        ax[0].grid('on')
        ax[0].set_xlabel('F [GHz]')
        ax[0].set_ylabel(str(nameof(señal)))   
        ax[0].set_title('Parte Real')

        ax[1].plot(f_norm,np.imag(Scal_fpos[i,:]))
        ax[1].grid('on')
        ax[1].set_xlabel('F [GHz]')
        ax[1].set_ylabel(str(nameof(señal)))
        ax[1].set_title('Parte Imag')

    return f_norm, Scal_fpos

In [66]:
F, S = transformada(S1_may, fc, Nfft, Ts)


ValueError: could not broadcast input array from shape (0,) into shape (1024,)